## Team Rocket Payment Prediction Model Notebook

In [1]:
# base imports
import pandas as pd
import numpy as np
import pymssql
import yaml
from yaml import Loader

In [2]:
# open yaml to get database connection strings
with open('secrets.yaml', 'r') as f:
    configs = yaml.load(f, Loader=Loader)

#### Connect to Database

In [3]:
# define database connection vars for future quer
server = configs['data']['server']
user = configs['data']['user']
password = configs['data']['password']
database = configs['data']['database']

# define table strings
efficiency_table = 'dbo.EfficiencyScores'
safety_table = 'dbo.SafetyScores'
outcomes_table = 'dbo.ClinicalOutcomeScores'
community_table = 'dbo.EngagementScores'
payment_table = 'dbo.PaymentAndValueOfCareVals'
state_table = 'dbo.States'
location_table = 'dbo.Locations'

try:
    # connect to database with above credentials
    conn = pymssql.connect(server, user, password, database)
    
    # instantiate cursor
    cursor = conn.cursor()
    
    # get efficiency data
    efficiency_query = f'SELECT * FROM {efficiency_table}'
    efficiency = pd.read_sql(efficiency_query, conn, index_col='Efficiency_ID')
    
    # get safety data
    safety_query = f'SELECT * FROM {safety_table}'
    safety = pd.read_sql(safety_query, conn, index_col='Safety_ID')
    
    # get outcomes data
    outcomes_query = f'SELECT * FROM {outcomes_table}'
    outcomes = pd.read_sql(outcomes_query, conn, index_col='ClinicalOutcome_ID')
    
    # get community data
    community_query = f'SELECT * FROM {community_table}'
    community = pd.read_sql(community_query, conn, 'EngagementScore_ID')
    
    #get payment data
    payment_query = f'SELECT * FROM {payment_table}'
    payment = pd.read_sql(payment_query, conn, index_col='Payment_ID')
    
    #get state data
    state_query = f'SELECT * FROM {state_table}'
    state = pd.read_sql(state_query, conn, index_col='State_ID')
    
    #get location data
    location_query = f'SELECT * FROM {location_table}'
    location = pd.read_sql(location_query, conn, index_col='Facility_ID')
except Exception as e:
    print(e)
    
print('done')

/home/charlie/m11-capstone/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/charlie/m11-capstone/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/charlie/m11-capstone/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/home/charlie/m11-capstone/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlch

done


/home/charlie/m11-capstone/env/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


The efficiency, outcomes, community and safety table are all the same size and contain the facility ID's that we have the corresponding data for. Notice the locations and payment table are much larger and contain facilities that other tables don't have data on. These rows will be dropped in the following merge.

In [4]:
joined = efficiency.merge(safety, on='Facility_ID', how='inner')
joined_1  = joined.merge(outcomes, on='Facility_ID', how='inner')
joined_2 = joined_1.merge(community, on='Facility_ID', how='inner')
joined_3 = joined_2.merge(location, on='Facility_ID', how='left')
joined_4 = joined_3.merge(state, on='State_ID', how='left')
final_join = payment.merge(joined_4, on='Facility_ID', how='inner')

We'll drop the folowing columns as they contain non-numeric, non-categorical data that would be no use to any of the models used below.

In [5]:
# drop uneeded columns
dropped = final_join.drop(['Facility_ID', 'Facility_Name', 'Location_City', 'Location_State', 'Location_Zip_Code', 'Location_County', 'State_Name'], axis=1)

The following variables were already label encoded but not with values that made sense in that I think the labels were standardized which isn't necessary. Below we redo this label encoding and followed by a one hot encoding

In [6]:
# label encode categorical columns appropriatelyl
dropped['Payment_Category'] = dropped['Payment_Category'].astype('category').cat.codes
dropped['Value_Of_Care_Category'] = dropped['Value_Of_Care_Category'].astype('category').cat.codes

In [7]:
# one hot encode the label encoded columns
pc_onehot = pd.get_dummies(dropped['Payment_Category'])
dropped = dropped.drop('Payment_Category', axis=1)
dropped = dropped.join(pc_onehot, lsuffix='_Pay')

vc_onehot = pd.get_dummies(dropped['Value_Of_Care_Category'])
dropped = dropped.drop('Value_Of_Care_Category', axis=1)
dropped = dropped.join(vc_onehot, rsuffix='_VC')

#### Payment Estimates Model

In [8]:
# ml imports
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import joblib

In [9]:
X = dropped[['Lower_Estimate', 'Higher_Estimate']]
y = dropped['Payment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lnrg = LinearRegression()
lnrg = lnrg.fit(X_train, y_train)
y_preds = lnrg.predict(X_test)
print(r2_score(y_test, y_preds))

0.9998155914442716


In [10]:
# saving/exporting model
estimates_model = joblib.dump(lnrg, 'estimates_model.model')

#### Linear Regression Payment Model

Started with a linear regression model to get a baseline of what to expect for regressions on this data. No cross validation was done.

In [11]:
X = dropped.drop(['Payment', 'Higher_Estimate', 'Lower_Estimate'], axis=1)
y = dropped['Payment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
lnrg = LinearRegression()
lnrg = lnrg.fit(X_train, y_train)
y_preds = lnrg.predict(X_test)
print(r2_score(y_test, y_preds))

0.6434151116740889


#### Lasso Regression Payment Model

After a pretty middling performance from our linear regression model we decided to exhaust the linear regression idea after trying just one more variant. The variant chosen was Lasso Regression. 

In [13]:
from sklearn.linear_model import Lasso

la_params = {'alpha': [0.0001,0.001,0.01,0.1,1,10,100]}

larg = Lasso()
la_grid = GridSearchCV(larg, la_params, scoring='r2', cv=5)
la_grid.fit(X_train, y_train)
y_preds = la_grid.predict(X_test)
print(r2_score(y_test, y_preds))

/home/charlie/m11-capstone/env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 105908017.92247444, tolerance: 356501.32587910385
  model = cd_fast.enet_coordinate_descent(
/home/charlie/m11-capstone/env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 42888634.16191161, tolerance: 358057.7856691777
  model = cd_fast.enet_coordinate_descent(
/home/charlie/m11-capstone/env/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 43247054.987890124, tolerance: 363547.952536348
  model = cd_fast.enet_coordinate_descent(
/home/charlie/m11-capstone/env/lib/python3.9/site-packages/

0.6420533626830127


The performance from this model was almost identical to that of the original linear regression at least in termns of its r-squared value which we are using in this case to be a general performance metric and quick comparison point between models

#### Random Forest Regressor Model

Moving over to ensemble methods we decided to first try a random forest regressor to see if we get any improvments over a standard regression. We support this with a bit of hyperparameter tuning with respect to the number of estimators, the maximum number of features, the mimimum number of samples to warrant a split and whether or not bootstrap samples are used.

In [14]:
from sklearn.ensemble import RandomForestRegressor

rf_params = { "n_estimators"      : [10,20,30],
              "max_features"      : ['sqrt', 'log2'],
              "min_samples_split" : [2,4,8],
              "bootstrap": [True, False],
            }

rfrg = RandomForestRegressor()
rf_grid = GridSearchCV(rfrg, rf_params, scoring='r2', cv=5)
rf_grid.fit(X_train, y_train)
y_preds = rf_grid.predict(X_test)
print(r2_score(y_test, y_preds))

0.6760349109986256


At first glance we see a noticeable improvement in the r-squared value of this model compared to the previous two. Ensemble methods seem to be getting more promising results.

#### XGBoost Model

What many consider to be the be-all and end-all of statistical models, XGBoost is the logical next step in maximizing ensemble regressors. This is supported by cross validation on the learning rate of the model, the maximum depth of each tree in the model, the minimum child weight of a node, the gamma of the model and the number of columns used in each tree.

In [15]:
from xgboost import XGBRegressor

xg_params = {'nthread':[4],
              'objective':['reg:squarederror'],
              'learning_rate': [.03, 0.05, .07],
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]
            }

'''
xg_params = {'learning_rate': (0.05, 0.10, 0.15),
             'max_depth': [3, 4, 5, 6, 8],
             'min_child_weight': [1, 3, 5, 7],
             'reg_alpha':[],
             'reg_lambda':[],
             'gamma':[0.0, 0.1, 0.2],
             'colsample_bytree':[0.3, 0.4],}
'''

xgrg = XGBRegressor()
xg_grid = GridSearchCV(xgrg, xg_params, scoring='r2', cv=2, n_jobs=5)
xg_grid.fit(X_train, y_train)
y_preds = xg_grid.predict(X_test)
print(r2_score(y_test, y_preds))

0.7830035159585188


This R-squared value is a marked improvement over the Random Forest and Linear regressors from above. Our model accounts for about 80% of the variance of the dependent variable which implies there is correlation to be found in this data.

In [16]:
print(mean_squared_error(y_test, y_preds))

585994.229443796


If we take a look at the mean squared error of the model we find the model has an average residual of about about $765. With respect to the payouts of medicare/medicaid for different procedures this is an acceptable error. Below you'll see the parameters used in the best performing version of our XGBoost model.

In [17]:
xg_grid.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.03,
 'max_depth': 5,
 'min_child_weight': 4,
 'n_estimators': 500,
 'nthread': 4,
 'objective': 'reg:squarederror',
 'subsample': 0.7}

In [18]:
# shippin' it
joblib.dump(xg_grid, 'xgboost_model.model')

['xgboost_model.model']